In [1]:
import numpy as np

Xdata = np.load("../data/6-parameter-maps/X_maps_Cosmogrid_100k.npy", allow_pickle=True)
Ydata = np.load("../data/6-parameter-maps/y_maps_Cosmogrid_100k.npy", allow_pickle=True)

num_samples = len(Xdata)
train_split, val_split, test_split = int(0.80*num_samples), \
            int(0.10*num_samples), int(0.10*num_samples) + 1

train_x, val_x, test_x = np.split(Xdata[:, :, :, None], [train_split, train_split+val_split])
train_y, val_y, test_y = np.split(Ydata, [train_split, train_split+val_split])

output_num = 5

train_y, val_y, test_y = train_y[:,:output_num], val_y[:,:output_num], test_y[:,:output_num]

print(f"x_train shape: {train_x.shape} - y_train shape: {train_y.shape}")
print(f"x_test shape: {test_x.shape} - y_test shape: {test_y.shape}")

x_train shape: (80000, 66, 66, 1) - y_train shape: (80000, 5)
x_test shape: (10000, 66, 66, 1) - y_test shape: (10000, 5)


In [2]:
# create jsonl files from these train, val, test splits
import jsonlines

for split_name, x, y in zip(['train', 'val', 'test'], [train_x, val_x, test_x], [train_y, val_y, test_y]):
    print(split_name)
    with jsonlines.open(f'../data/6-parameter-maps/{split_name}.jsonl', mode='w') as writer:
        for i in range(len(x)):
            Omega_m, H_0, n_s, sigma_8, w_0 = y[i].tolist()
            if list(map(type, [Omega_m, H_0, n_s, sigma_8, w_0])) != [float]*5:
                print(Omega_m, H_0, n_s, sigma_8, w_0)
            writer.write({'map': x[i].tolist(), 
                            'Omega_m': float(Omega_m), 'H_0': float(H_0), 'n_s': float(n_s), 
                            'sigma_8': float(sigma_8), 'w_0': float(w_0)})
            # break
    # break


train
val
test


# my maps

In [2]:
import numpy as np
import os, sys
import pandas as pd

In [3]:
data_dir = "../data/20230814_224x224/"
maps = os.listdir(data_dir)
for m in maps:
    print(m)
    d = np.load(os.path.join(data_dir, m))
    print(d.shape)
    break

cosmo_099934.npy
(224, 224, 224, 4)


In [4]:
df = pd.read_csv("../data/parameters.csv")
df

,sim_type,sim_name,As,bary_Mc,bary_nu,H0,O_cdm,O_nu,Ob,Om,ns,s8,w0
0,fiducial,cosmo_fiducial,3.058901e-09,6.600000e+13,0.000000,67.360000,0.209277,0.001423,0.049300,0.260000,0.964900,0.840000,-1.000000
1,fiducial,cosmo_delta_H0_m,3.283237e-09,6.600000e+13,0.000000,65.360000,0.209189,0.001511,0.049300,0.260000,0.964900,0.840000,-1.000000
2,fiducial,cosmo_delta_H0_p,2.857256e-09,6.600000e+13,0.000000,69.360000,0.209358,0.001342,0.049300,0.260000,0.964900,0.840000,-1.000000
3,fiducial,cosmo_delta_Ob_m,3.014249e-09,6.600000e+13,0.000000,67.360000,0.210277,0.001423,0.048300,0.260000,0.964900,0.840000,-1.000000
4,fiducial,cosmo_delta_Ob_p,3.104330e-09,6.600000e+13,0.000000,67.360000,0.208277,0.001423,0.050300,0.260000,0.964900,0.840000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,grid,cosmo_204707,5.508026e-09,6.800107e+14,1.927475,65.988457,0.171894,0.001482,0.048556,0.221931,0.999455,0.999483,-1.102559
2517,benchmarks,box_size,3.058901e-09,6.600000e+13,0.000000,67.360000,0.209277,0.001423,0.049300,0.260000,0.964900,0.840000,-1.000000
2518,benchmarks,fiducial_bench,3.058901e-09,6.600000e+13,0.000000,67.360000,0.209277,0.001423,0.049300,0.260000,0.964900,0.840000,-1.000000
2519,benchmarks,particle_count,3.058901e-09,6.600000e+13,0.000000,67.360000,0.209277,0.001423,0.049300,0.260000,0.964900,0.840000,-1.000000


In [5]:
df[df["sim_name"] == "cosmo_000001"]

,sim_type,sim_name,As,bary_Mc,bary_nu,H0,O_cdm,O_nu,Ob,Om,ns,s8,w0
17,grid,cosmo_000001,2.019919e-09,3.162278e+13,0.0,73.0,0.253789,0.001211,0.045,0.3,0.97,0.9,-1.1665


In [7]:
import tqdm

data_dir = "../data/20230814_224x224/"
maps = os.listdir(data_dir)

num_saved = 0
counter = train_split
split_name = "train"
writer = open(f'{data_dir}/{split_name}.txt', mode='w')

for m in tqdm.tqdm(maps):
    if "npy" not in m:
        # print("skipping", m)
        continue
    # if m == "cosmo_193238.npy":
    #     # print("skipping", m)
    #     continue
    d = np.load(os.path.join(data_dir, m))
    if d.shape[0] == 0:
        # print("Skipping", m)
        continue
    row = df[df["sim_name"] == m[:-4]]
    writer.write(m + "\n")
    
    num_saved += 1
    if num_saved == counter:
        print(split_name)
        num_saved = 0
        writer.close()
        if split_name == "train":
            counter = val_split
            split_name = "val"
            writer.close()
            writer = open(f'{data_dir}/{split_name}.txt', mode='w')
        elif split_name == "val":
            counter = test_split
            split_name = "test"
            writer.close()
            writer = open(f'{}/{split_name}.txt', mode='w')
        print(split_name)
    # break
writer.close()

NameError: name 'train_split' is not defined

In [20]:
writer.close()

In [8]:
import tqdm

all_x, all_y = np.empty((0, 224, 224, 4)), np.empty((0, 13))
data_dir = "../data/20230814_224x224/"
maps = os.listdir(data_dir)

for m in tqdm.tqdm(maps[:1]):
    d = np.load(os.path.join(data_dir, m))
    row = df[df["sim_name"] == m[:-4]]
    all_x = np.append(all_x, d, axis=0)
    all_y = np.append(all_y, np.tile(row, (d.shape[0], 1)), axis=0)
    
all_x.shape, all_y.shape

100%|██████████| 1/1 [00:00<00:00,  5.99it/s]


((140, 224, 224, 4), (140, 13))

In [10]:
import tqdm

num_patches = 32
all_x, all_y = np.empty((0, 224, 224, 4)), np.empty((0, 13))
data_dir = "../data/20230814_224x224/"
maps = os.listdir(data_dir)

num_maps = 0
num_cosmo = 0

for m in tqdm.tqdm(maps):
    if ".npy" not in m:
        print(m)
        continue
    d = np.load(os.path.join(data_dir, m))
    num_maps += d.shape[0]
    num_cosmo += (d.shape[0] != 0)
    if d.shape[0] not in [0, num_patches*7]:
        print(m, d.shape[0])
    
num_maps, num_cosmo

 11%|█         | 277/2504 [01:07<08:47,  4.22it/s]

train.txt


 55%|█████▍    | 1372/2504 [06:51<06:46,  2.79it/s]

val.txt


 59%|█████▉    | 1472/2504 [07:36<05:24,  3.18it/s]

...

test.txt


 79%|███████▉  | 1988/2504 [10:25<03:03,  2.81it/s]

20230814_224x224.py


100%|██████████| 2504/2504 [14:39<00:00,  2.85it/s]


(542912, 2424)

In [13]:
num_maps = 2424
train_split, val_split, test_split = int(0.80*num_maps), \
                int(0.10*num_maps), int(0.10*num_maps)

print(train_split, val_split, test_split, train_split+val_split+test_split)

1939 242 242 2423


In [ ]:
# create jsonl files from these train, val, test splits
import jsonlines
import tqdm

data_dir = "../data/20230419_224x224/"
maps = os.listdir(data_dir)

num_saved = 0
counter = train_split
split_name = "train"
writer = jsonlines.open(f'../data/20230419_224x224/{split_name}.jsonl', mode='w')

for m in tqdm.tqdm(maps):
    if m == "cosmo_193238.npy":
        print("skipping", m)
    d = np.load(os.path.join(data_dir, m))
    if d.shape[0] == 0:
        print("Skipping", m)
    row = df[df["sim_name"] == m[:-4]]
    sim_type, sim_name, As, bary_Mc, bary_nu, H0, O_cdm, O_nu, Ob, Om, ns, s8, w0 = row.values[0]
    writer.write({'map': d.tolist(), 'sim_type': str(sim_type), 'sim_name': str(sim_name),
                  'As': float(As), 'bary_Mc': float(bary_Mc), 'bary_nu': float(bary_nu), 'H0': float(H0), 
                  'O_cdm': float(O_cdm), 'O_nu': float(O_nu), 'Ob': float(Ob), 'Om': float(Om), 
                  'ns': float(ns), 's8': float(s8), 'w0': float(w0)
                 })
    
    num_saved += 1
    if num_saved == counter:
        print(split_name)
        num_saved = 0
        writer.close()
        if split_name == "train":
            counter = val_split
            split_name = "val"
            writer = jsonlines.open(f'../data/20230419_224x224/{split_name}.jsonl', mode='w')
        if split_name == "val":
            counter = test_split
            split_name = "test"
            writer = jsonlines.open(f'../data/20230419_224x224/{split_name}.jsonl', mode='w')
        print(split_name)
    # break

  0%|          | 8/2501 [01:12<6:03:22,  8.75s/it]

Skipping cosmo_202679.npy


  1%|▏         | 32/2501 [04:31<5:59:34,  8.74s/it]

Skipping cosmo_193523.npy


  5%|▍         | 113/2501 [16:15<5:40:45,  8.56s/it]

...

 42%|████▏     | 1039/2501 [2:38:41<3:47:09,  9.32s/it]

Skipping cosmo_203054.npy


 42%|████▏     | 1057/2501 [2:41:30<3:57:21,  9.86s/it]

Skipping cosmo_200327.npy


 43%|████▎     | 1067/2501 [2:43:00<3:55:07,  9.84s/it]

In [13]:
import jsonlines
import tqdm

split_name = "train"

a = jsonlines.open(f'../data/20230419_224x224/{split_name}.jsonl', mode='r')
num = 0
for line in a:
    if num % 1 == 0:
        print(num)
    # print(line.keys())
    # print("here")
    num+=1
    # break
a.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218


KeyboardInterrupt: 

In [90]:
np.array(line["map"]).shape

(70, 224, 224, 4)

In [ ]:
num